<a id=contents></a>

# Model building
## What are we trying to predict?


[1. ETL and Train Test Split](#ETL)

[2. Modelling with Random Forest](#RF)

[3. Modelling with Conditional Random Field](#CRF)

[4. Choice of model architectures](#selection)

[4.1 Model 1](#one)

[4.2 Model 2](#two)

[4.2 Model 3](#three)

[4.2 Model 4](#four)

[4.2 Model 5](#five)

[7. Conclusions and model comparison table](#conc)

In [35]:

import pandas as pd
import numpy as np

import pickle

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set_style("darkgrid")

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate
from matplotlib import cm
import numpy as np
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
from sklearn.metrics import f1_score

from sklearn_crfsuite import CRF, scorers, metrics
from sklearn.model_selection import cross_val_predict
from sklearn_crfsuite.metrics import flat_classification_report
from sklearn.metrics import classification_report, make_scorer

from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import re
import string
tokenizer = RegexpTokenizer(r'\b\w{3,}\b')
stop_words = list(set(stopwords.words("english")))
stop_words += list(string.punctuation)

import warnings
warnings.filterwarnings('ignore')

from scipy import stats as ss

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<a id=ETL ><a/> 

## 1. ETL of data and Train-Test Split
    
[LINK to table of contents](#contents)

In [11]:
feature_df = pd.read_pickle('feature_based_data/final_feature_data.pkl')
feature_y = feature_df.Tag
feature_X = feature_df.drop(columns=['Tag'])

x_train, x_test, y_train, y_test = train_test_split(feature_X, feature_y, test_size=.2, random_state=12345)

In [58]:
x_train.head(2)

,is_title,length,is_upper,is_digit,is_prev_NE,prev_-1_POS_NNP,prev_-1_POS_NN,is_prev_pos_same_as_current,POS_NNP,POS_NN,POS_IN,POS_DT,POS_.
42758,0,7,0,0,0,0,0,0,0,1,0,0,0
56828,0,4,0,0,0,0,0,0,0,0,0,0,0


<a id = 'RF'></a>

## 2. Modelling with Random Forest

[LINK to table of contents](#contents)

In [46]:
%time
RF = RandomForestClassifier(n_estimators=50, max_depth=20, min_samples_split=.01, n_jobs=-1)

preds = cross_val_predict(estimator=RF, X=x_train.values, y=y_train.values, cv=5)


CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 12.2 µs


In [50]:
preds.shape == y_train.shape

True

In [53]:
report = classification_report(y_pred=preds, y_true=y_train)
print(report)

              precision    recall  f1-score   support

       B-art       0.00      0.00      0.00        37
       B-eve       0.00      0.00      0.00        34
       B-geo       0.44      0.90      0.59      1668
       B-gpe       0.59      0.57      0.58       973
       B-nat       0.00      0.00      0.00        17
       B-org       0.59      0.21      0.31       987
       B-per       0.52      0.34      0.41       907
       B-tim       0.83      0.20      0.32       926
       I-art       0.00      0.00      0.00        26
       I-eve       0.00      0.00      0.00        33
       I-geo       0.49      0.12      0.20       331
       I-gpe       0.00      0.00      0.00        27
       I-nat       0.00      0.00      0.00         9
       I-org       0.42      0.21      0.28       744
       I-per       0.50      0.92      0.65       961
       I-tim       0.98      0.36      0.52       261
           O       0.98      0.99      0.99     44987

    accuracy              

In [83]:
y_train_bin = np.zeros(y_train.shape)

In [84]:
for i in y_train_bin:
    if y_train[i]!='O':
        y_train_bin[i]=1
y_train_bin

array([0., 0., 0., ..., 0., 0., 0.])

In [15]:
x_train.iloc[:5].to_dict()

{'is_title': {42758: 0, 56828: 0, 18522: 0, 15552: 0, 20990: 0},
 'length': {42758: 7, 56828: 4, 18522: 1, 15552: 2, 20990: 6},
 'is_upper': {42758: 0, 56828: 0, 18522: 0, 15552: 0, 20990: 0},
 'is_digit': {42758: 0, 56828: 0, 18522: 0, 15552: 0, 20990: 0},
 'is_prev_NE': {42758: 0, 56828: 0, 18522: 1, 15552: 0, 20990: 1},
 'prev_-1_POS_NNP': {42758: 0, 56828: 0, 18522: 1, 15552: 0, 20990: 1},
 'prev_-1_POS_NN': {42758: 0, 56828: 0, 18522: 0, 15552: 0, 20990: 0},
 'is_prev_pos_same_as_current': {42758: 0,
  56828: 0,
  18522: 0,
  15552: 0,
  20990: 0},
 'POS_NNP': {42758: 0, 56828: 0, 18522: 0, 15552: 0, 20990: 0},
 'POS_NN': {42758: 1, 56828: 0, 18522: 0, 15552: 0, 20990: 0},
 'POS_IN': {42758: 0, 56828: 0, 18522: 0, 15552: 1, 20990: 0},
 'POS_DT': {42758: 0, 56828: 0, 18522: 0, 15552: 0, 20990: 0},
 'POS_.': {42758: 0, 56828: 0, 18522: 1, 15552: 0, 20990: 0}}

<a id = 'CRF'></a>

## 3. Modelling with a Conditional Random Field

[LINK to table of contents](#contents)

Sklearn's CRF requires the input data to be a list of lists of dicts. I stored these as pickle files in notebook 3.

In [23]:
with open('clean_data/crf_train_data.pkl', 'rb') as f:
    crf_features_train = pickle.load(f)
    
with open('clean_data/crf_test_data.pkl', 'rb') as f:
    crf_features_test = pickle.load(f)
    
with open('clean_data/crf_train_targets.pkl', 'rb') as f:
    crf_targets_train = pickle.load(f)
    
with open('clean_data/crf_test_targets.pkl', 'rb') as f:
    crf_targets_test = pickle.load(f)
    

In [24]:
crf = CRF(algorithm='lbfgs',
          c1=0.1,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=True)

In [25]:
%time
crf.fit(crf_features_train, crf_targets_train)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 4.05 µs


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [26]:
labels = list(crf.classes_)
labels

['O',
 'B-geo',
 'B-gpe',
 'B-per',
 'I-geo',
 'B-org',
 'I-org',
 'B-tim',
 'B-art',
 'I-art',
 'I-per',
 'I-gpe',
 'I-tim',
 'B-nat',
 'B-eve',
 'I-eve',
 'I-nat']

In [39]:
crf_y_pred_train = crf.predict(crf_features_train)
metrics.flat_f1_score(crf_targets_train, crf_y_pred_train,
                      average='weighted', labels=labels)

0.9852280959671684

In [33]:
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0]))

print(metrics.flat_classification_report(
    crf_targets_train, crf_y_pred_train, labels=sorted_labels, digits=3))

              precision    recall  f1-score   support

           O      0.996     0.998     0.997     45078
       B-art      0.932     0.804     0.863        51
       I-art      0.941     1.000     0.970        32
       B-eve      0.974     0.949     0.961        39
       I-eve      1.000     0.939     0.969        33
       B-geo      0.849     0.936     0.890      1596
       I-geo      0.891     0.920     0.906       339
       B-gpe      0.915     0.853     0.883      1026
       I-gpe      0.895     0.531     0.667        32
       B-nat      1.000     0.944     0.971        18
       I-nat      1.000     1.000     1.000         9
       B-org      0.921     0.816     0.865      1010
       I-org      0.949     0.951     0.950       721
       B-per      0.964     0.967     0.965       839
       I-per      0.973     0.988     0.981       995
       B-tim      0.980     0.897     0.937       923
       I-tim      0.948     0.890     0.918       246

    accuracy              

In [37]:
crf_y_pred_test = crf.predict(crf_features_test)
metrics.flat_f1_score(crf_targets_test, crf_y_pred_test,
                      average='weighted', labels=labels)

0.9471352834210373

In [38]:
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0]))

print(metrics.flat_classification_report(
    crf_targets_test, crf_y_pred, labels=sorted_labels, digits=3))

              precision    recall  f1-score   support

           O      0.984     0.987     0.986     11139
       B-art      0.000     0.000     0.000         2
       I-art      0.000     0.000     0.000         2
       B-eve      0.400     0.333     0.364         6
       I-eve      0.400     0.500     0.444         4
       B-geo      0.769     0.738     0.753       474
       I-geo      0.600     0.600     0.600        75
       B-gpe      0.788     0.819     0.803       204
       I-gpe      0.000     0.000     0.000         2
       B-nat      0.000     0.000     0.000         2
       I-nat      0.000     0.000     0.000         0
       B-org      0.628     0.626     0.627       227
       I-org      0.728     0.639     0.681       205
       B-per      0.805     0.709     0.754       268
       I-per      0.689     0.908     0.783       239
       B-tim      0.887     0.793     0.837       237
       I-tim      0.754     0.591     0.662        88

   micro avg      0.948   

In [41]:
import eli5

In [42]:
eli5.show_weights(crf, top=30)
# repeat this for the hyperparam optimised version

From \ To,O,B-art,I-art,B-eve,I-eve,B-geo,I-geo,B-gpe,I-gpe,B-nat,I-nat,B-org,I-org,B-per,I-per,B-tim,I-tim
O,3.34,0.796,-2.118,2.335,-1.988,1.232,-4.234,0.638,-1.858,0.629,-1.87,1.313,-4.505,1.884,-3.625,1.916,-4.072
B-art,-0.321,0.0,4.978,0.0,0.0,-0.658,-0.708,-1.079,-0.048,0.0,0.0,-0.588,-0.943,-0.94,-1.402,0.181,-0.277
I-art,-1.181,-0.001,4.767,0.0,0.0,0.359,-0.537,-0.557,0.0,0.0,0.0,-0.309,-0.73,-0.857,-1.104,0.0,-0.065
B-eve,-1.053,0.0,0.0,0.0,4.506,-0.668,-0.615,-0.941,-0.303,0.0,0.0,-0.6,-1.242,-0.698,-0.861,0.213,-0.384
I-eve,-0.084,0.0,0.0,-0.609,3.942,-0.094,-0.152,-0.313,0.0,0.0,0.0,-0.252,-0.267,-0.221,-1.065,-0.136,0.0
B-geo,0.505,0.404,-0.887,-0.59,-1.238,-2.332,3.828,-0.614,-2.201,0.0,-0.442,-1.706,-2.941,-3.045,-2.945,1.298,-1.725
I-geo,0.142,-0.003,-0.187,-0.155,-0.012,-2.038,3.402,-2.009,-0.362,0.0,-0.061,-0.85,-1.96,-1.274,-1.487,0.321,-0.815
B-gpe,0.385,-0.824,-1.182,-0.084,-1.448,-2.31,-2.814,-3.249,2.958,-0.126,-0.494,0.715,-3.356,0.135,-2.429,-2.775,-1.81
I-gpe,-0.024,0.0,0.0,0.0,0.0,0.492,-0.029,-0.175,4.281,0.0,0.0,-0.155,-0.379,-0.307,-0.49,0.0,0.0
B-nat,-0.17,0.0,0.0,0.0,0.0,-0.335,0.0,-0.366,0.0,0.0,3.752,-0.145,-0.354,-0.162,-0.321,0.0,0.0


### Classification Report Interpretation for test data:

The main reported figure is the weighted average F1 Score:

$$        \frac{2 * Precision * Recall}{Precision + Recall}         $$

The support column refers to how many instances there are of each class. As we've seen before, this distribution is dominated by 'O' (non-NE) and there are some, such as B-nat ('national entity') that are almost zero (there were only 18 instances in the training data). 



In [47]:
crf_optim = CRF(algorithm='lbfgs',
          max_iterations=100,
          all_possible_transitions=True)

f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=labels)

crf_params = {'c1': [0.05, 0.1, 0.5, 1.0, 1.5,  2.0],
              'c2': [0.05, 0.1,  0.5, 1.0, 1.5,  2.0]}

grid =  GridSearchCV(crf_optim, 
                    crf_params, 
                    f1_scorer, 
                    -1, cv=5, 
                    return_train_score=True, 
                    verbose=True)

grid.fit(crf_features_train, crf_targets_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed: 58.4min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=CRF(algorithm='lbfgs', all_possible_states=None,
                           all_possible_transitions=True, averaging=None,
                           c=None, c1=None, c2=None,
                           calibration_candidates=None, calibration_eta=None,
                           calibration_max_trials=None, calibration_rate=None,
                           calibration_samples=None, delta=None, epsilon=None,
                           error_sensitive=None, gamma=None,
                           keep_tempfi...
             iid='deprecated', n_jobs=-1,
             param_grid={'c1': [0.05, 0.1, 0.5, 1.0, 1.5, 2.0],
                         'c2': [0.05, 0.1, 0.5, 1.0, 1.5, 2.0]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
             scoring=make_scorer(flat_f1_score, average=weighted, labels=['O', 'B-geo', 'B-gpe', 'B-per', 'I-geo', 'B-org', 'I-org', 'B-tim', 'B-art', 'I-art', 'I-per', 'I-gpe', 'I

In [48]:
best_crf = grid.best_estimator_

In [49]:
best_crf.get_params()

{'algorithm': 'lbfgs',
 'all_possible_states': None,
 'all_possible_transitions': True,
 'averaging': None,
 'c': None,
 'c1': 0.05,
 'c2': 0.5,
 'calibration_candidates': None,
 'calibration_eta': None,
 'calibration_max_trials': None,
 'calibration_rate': None,
 'calibration_samples': None,
 'delta': None,
 'epsilon': None,
 'error_sensitive': None,
 'gamma': None,
 'keep_tempfiles': None,
 'linesearch': None,
 'max_iterations': 100,
 'max_linesearch': None,
 'min_freq': None,
 'model_filename': None,
 'num_memories': None,
 'pa_type': None,
 'period': None,
 'trainer_cls': None,
 'variance': None,
 'verbose': False}

In [51]:
# further fine tuning around the c1 penalty term
crf_optim = CRF(algorithm='lbfgs',
          max_iterations=100,
          all_possible_transitions=True)

f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=labels)

crf_params = {'c1': [0.01, 0.02, 0.03, 0.04, 0.05,],
              'c2': [ 0.45, 0.5, 0.55]}

grid =  GridSearchCV(crf_optim, 
                    crf_params, 
                    f1_scorer, 
                    -1, cv=5, 
                    return_train_score=True, 
                    verbose=True)

grid.fit(crf_features_train, crf_targets_train)

best_crf = grid.best_estimator_

Fitting 5 folds for each of 15 candidates, totalling 75 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed:  7.8min finished


In [53]:
best_crf.get_params()

{'algorithm': 'lbfgs',
 'all_possible_states': None,
 'all_possible_transitions': True,
 'averaging': None,
 'c': None,
 'c1': 0.04,
 'c2': 0.45,
 'calibration_candidates': None,
 'calibration_eta': None,
 'calibration_max_trials': None,
 'calibration_rate': None,
 'calibration_samples': None,
 'delta': None,
 'epsilon': None,
 'error_sensitive': None,
 'gamma': None,
 'keep_tempfiles': None,
 'linesearch': None,
 'max_iterations': 100,
 'max_linesearch': None,
 'min_freq': None,
 'model_filename': None,
 'num_memories': None,
 'pa_type': None,
 'period': None,
 'trainer_cls': None,
 'variance': None,
 'verbose': False}

In [54]:
# further fine tuning around the c1 penalty term
crf_optim = CRF(algorithm='lbfgs',
          max_iterations=100,
          all_possible_transitions=True)

f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=labels)

crf_params = {'c1': [0.0375, 0.04, 0.0425,],
              'c2': [ 0.425, 0.45, 0.475]}

grid =  GridSearchCV(crf_optim, 
                    crf_params, 
                    f1_scorer, 
                    -1, cv=5, 
                    return_train_score=True, 
                    verbose=True)

grid.fit(crf_features_train, crf_targets_train)

best_crf = grid.best_estimator_

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:  4.5min finished


In [60]:
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0]))

crf_y_test_pred = best_crf.predict(crf_features_test)

print(metrics.flat_classification_report(
    crf_targets_test, crf_y_test_pred, labels=sorted_labels, digits=3))

              precision    recall  f1-score   support

           O      0.983     0.989     0.986     11139
       B-art      0.000     0.000     0.000         2
       I-art      0.000     0.000     0.000         2
       B-eve      0.500     0.333     0.400         6
       I-eve      0.333     0.250     0.286         4
       B-geo      0.771     0.768     0.770       474
       I-geo      0.568     0.613     0.590        75
       B-gpe      0.822     0.814     0.818       204
       I-gpe      0.000     0.000     0.000         2
       B-nat      0.000     0.000     0.000         2
       I-nat      0.000     0.000     0.000         0
       B-org      0.668     0.612     0.639       227
       I-org      0.747     0.634     0.686       205
       B-per      0.809     0.709     0.755       268
       I-per      0.687     0.908     0.782       239
       B-tim      0.917     0.797     0.853       237
       I-tim      0.828     0.545     0.658        88

   micro avg      0.950   

In [57]:
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0]))

crf_y_train_pred = best_crf.predict(crf_features_train)

print(metrics.flat_classification_report(
    crf_targets_train, crf_y_train_pred, labels=sorted_labels, digits=3))

              precision    recall  f1-score   support

           O      0.993     0.997     0.995     45078
       B-art      0.970     0.627     0.762        51
       I-art      1.000     0.844     0.915        32
       B-eve      0.941     0.821     0.877        39
       I-eve      0.939     0.939     0.939        33
       B-geo      0.820     0.934     0.873      1596
       I-geo      0.858     0.906     0.881       339
       B-gpe      0.911     0.822     0.864      1026
       I-gpe      0.909     0.312     0.465        32
       B-nat      1.000     0.500     0.667        18
       I-nat      1.000     0.667     0.800         9
       B-org      0.909     0.768     0.833      1010
       I-org      0.922     0.936     0.929       721
       B-per      0.946     0.933     0.939       839
       I-per      0.942     0.987     0.964       995
       B-tim      0.971     0.862     0.913       923
       I-tim      0.912     0.760     0.829       246

    accuracy              

So we have been able to achieve a weighted average F1 score of **0.949 on the test data** (0.980 on train data respectively). Therefore there is a small train to test drop in performance, which looks greater when you compare the macro average F1 (test : 0.484; train : 0.850). 

### Assessing results with seqeval

So far I've been relying on Sklearn's standard metrics library, however this evaluation leaves a great deal of room for error. For one thing, it considers the B- and I- tags wholly distinct and does not help with pairing them the way they should be pared into single, whole entities. So I am going to use the python library seqeval, which has been specifically designed to work with BIO labels and to help with measuring performance on tasks "[such as named-entity recognition, part-of-speech tagging, semantic role labeling](#https://pypi.org/project/seqeval/)".

In [63]:
from seqeval.metrics import accuracy_score as seq_acc
from seqeval.metrics import classification_report as seq_cr
from seqeval.metrics import f1_score as seq_f1_score


In [73]:
print("Our overall macro-average F1 score is", round(seq_f1_score(crf_targets_test, crf_y_test_pred, average='micro'),3))

Our overall macro-average F1 score is 0.729


In [76]:
print("Our overall accuracy is", round(seq_acc(crf_targets_test, crf_y_test_pred),4),)

Our overall accuracy is 0.9497


As you'd expect, the results are very different compared to sklearn's estimation, however this is a much more realistic picture of how well our model is performing. The model is pulled down considerably by the low-frequency classes of 

In [69]:
print(seq_cr(crf_targets_test, crf_y_test_pred))

             precision    recall  f1-score   support

        geo       0.76      0.76      0.76       474
        gpe       0.82      0.81      0.82       204
        tim       0.85      0.74      0.79       237
        org       0.65      0.59      0.62       227
        per       0.69      0.61      0.65       268
        eve       0.50      0.33      0.40         6
        art       0.00      0.00      0.00         2
        nat       0.00      0.00      0.00         2

avg / total       0.75      0.71      0.73      1420



In [56]:
# And our accuracy score
best_crf.score(crf_features_train, crf_targets_train)

0.9802593088870855

<a id=ttsplit ><a/> 

## 3. Investigating our best model's weights
   
[LINK to table of contents](#contents)

In [59]:
eli5.show_weights(best_crf, top=30)


From \ To,O,B-art,I-art,B-eve,I-eve,B-geo,I-geo,B-gpe,I-gpe,B-nat,I-nat,B-org,I-org,B-per,I-per,B-tim,I-tim
O,3.374,1.116,-1.561,2.038,-1.253,1.351,-2.744,1.076,-1.165,0.965,-1.092,1.475,-2.792,2.373,-2.162,2.223,-2.721
B-art,-0.373,-0.043,3.926,-0.023,-0.112,-0.524,-0.384,-0.681,-0.12,-0.032,-0.059,-0.47,-0.58,-0.56,-0.536,0.164,-0.329
I-art,-0.986,-0.039,3.672,-0.006,-0.075,0.122,-0.277,-0.42,-0.059,0.0,-0.019,-0.319,-0.392,-0.409,-0.659,0.0,-0.207
B-eve,-0.929,-0.027,-0.109,-0.062,3.892,-0.523,-0.401,-0.553,-0.119,0.0,-0.065,-0.436,-0.62,-0.467,-0.484,0.162,-0.238
I-eve,-0.153,0.0,-0.041,-0.356,2.923,-0.181,-0.188,-0.187,-0.037,0.0,0.0,-0.21,-0.234,-0.132,-0.41,-0.136,-0.112
B-geo,0.517,0.175,-0.742,-0.353,-0.795,-1.523,3.769,-0.482,-0.874,-0.12,-0.466,-1.171,-1.649,-1.747,-1.545,1.3,-1.025
I-geo,0.088,-0.162,-0.362,-0.126,-0.211,-1.168,3.103,-0.916,-0.315,-0.019,-0.123,-0.645,-0.909,-0.849,-1.084,0.242,-0.526
B-gpe,0.477,-0.439,-0.713,-0.191,-0.795,-1.39,-1.581,-1.923,2.989,-0.153,-0.422,0.9,-1.856,0.891,-1.203,-1.453,-1.01
I-gpe,-0.327,-0.003,-0.022,0.0,-0.038,0.205,-0.129,-0.206,2.617,0.0,0.0,-0.232,-0.287,-0.29,-0.302,-0.105,-0.119
B-nat,-0.474,0.0,-0.022,0.0,-0.068,-0.269,-0.157,-0.368,-0.015,-0.023,2.562,-0.265,-0.24,-0.265,-0.314,-0.109,-0.101


<a id=selection></a>

## 4. Choice of model architectures

[LINK to table of contents](#contents)

<a id=one ><a/> 

## 4.1 Model 1
    
[LINK to table of contents](#contents)

<a id=two ><a/> 

## 4.2 Model 2
    
[LINK to table of contents](#contents)

<a id=three ><a/> 

## 4.3 Model 3
    
[LINK to table of contents](#contents)

<a id=four ><a/> 

## 4.4 Model 4
    
[LINK to table of contents](#contents)

<a id=five ><a/> 

## 4.5 Model 5
    
[LINK to table of contents](#contents)

<a id=conc ><a/> 

## 7. Conclusions and model comparison table
    
[LINK to table of contents](#contents)